In [1]:

import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
import plotly.express as px
from imblearn.under_sampling import TomekLinks

In [2]:


def encode_categorical_data(dados, categoricos):
    dados_copy = dados.copy()
    encoder = OneHotEncoder(drop='if_binary')
    df_encoded = pd.DataFrame(encoder.fit_transform(dados_copy[categoricos]).toarray(),
                              columns=encoder.get_feature_names_out(categoricos))
    df_final = pd.concat([dados_copy.drop(categoricos, axis=1), df_encoded], axis=1)
    return df_final

def preprocess_data(df_final):
    X = df_final.drop('convertido', axis=1)
    y = df_final['convertido']
    
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)
    
    x_treino, x_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)
    
    scaler = StandardScaler()
    x_treino = scaler.fit_transform(x_treino)
    x_teste = scaler.transform(x_teste)
    
    smote = SMOTE(random_state=42)
    x_treino_resampled, y_treino_resampled = smote.fit_resample(x_treino, y_treino)
    
    return x_treino_resampled, y_treino_resampled, x_teste, y_teste

def train_xgboost_model(x_treino_resampled, y_treino_resampled):
    modelo_xgboost = XGBClassifier(objective='binary:logistic', use_label_encoder=False, eval_metric='logloss', reg_alpha=0.001, reg_lambda=0.001)
    modelo_xgboost.fit(x_treino_resampled, y_treino_resampled)
    return modelo_xgboost

def evaluate_model(modelo_xgboost, x_teste, y_teste):
    probabilidades_teste = modelo_xgboost.predict_proba(x_teste)[:, 1]
    y_pred_probs = modelo_xgboost.predict_proba(x_teste)[:, 1]
    y_pred_classes = np.round(y_pred_probs)
    acuracia = accuracy_score(y_teste, y_pred_classes)
    print(f'Acurácia no conjunto de teste: {acuracia * 100:.2f}%')

def generate_results_dataframe(dados, x_teste, y_teste, probabilidades_teste):
    df_temporario_teste = pd.DataFrame({
        'nm_prop_oportunidade': dados.loc[x_teste.index, 'nm_prop_oportunidade'],
        'convertido': y_teste,
        'probabilidade_conversao': probabilidades_teste
    })

    df_temporario_teste = df_temporario_teste.drop_duplicates(subset=['nm_prop_oportunidade', 'convertido'])

    df_resultados_teste = df_temporario_teste.groupby('nm_prop_oportunidade', as_index=False).agg({
        'convertido': 'first',
        'probabilidade_conversao': 'mean'
    })

    with pd.option_context('display.max_rows', None):
        display(df_resultados_teste[['nm_prop_oportunidade', 'convertido', 'probabilidade_conversao']])
    
    pd.reset_option('display.max_rows')

    df_resultados_teste.to_csv('resultados_teste_xgboost.csv', index=False)

dados = pd.read_csv(
    r"D:\tensorflowteste\excelfiles\export1.csv",
    sep=",")

categoricos = ['id_quote','dt_criacao','ds_tipo_obra','nm_prop_oportunidade',
               'nm_ambiente','promocional','tipo_loja']

df_final = encode_categorical_data(dados, categoricos)
x_treino_resampled, y_treino_resampled, x_teste, y_teste = preprocess_data(df_final)
modelo_xgboost = train_xgboost_model(x_treino_resampled, y_treino_resampled)
evaluate_model(modelo_xgboost, x_teste, y_teste)
probabilidades_teste = modelo_xgboost.predict_proba(x_teste)[:, 1]
generate_results_dataframe(dados, pd.DataFrame(x_teste, columns=df_final.drop('convertido', axis=1).columns), y_teste, probabilidades_teste)



Acurácia no conjunto de teste: 73.77%


,nm_prop_oportunidade,convertido,probabilidade_conversao
0,ana paula goncalves lemos,1,0.354008
1,fabiano martins werutsky,0,0.489150
2,juliana westphalen gonçalves,1,0.366401


In [3]:
from functools import wraps

def resultados_decorator(funcao):
    @wraps(funcao)
    def wrapper(*args, **kwargs):
        
        dados, categoricos = args

     
        df_final = encode_categorical_data(dados, categoricos)
        x_treino_resampled, y_treino_resampled, x_teste, y_teste = preprocess_data(df_final)

        
        modelo_xgboost = train_xgboost_model(x_treino_resampled, y_treino_resampled)

        
        evaluate_model(modelo_xgboost, x_teste, y_teste)
        probabilidades_teste = modelo_xgboost.predict_proba(x_teste)[:, 1]

     
        generate_results_dataframe(dados,
                                    pd.DataFrame(x_teste, columns=df_final.drop('convertido',
                  axis=1).columns), y_teste, probabilidades_teste)

    return wrapper


@resultados_decorator
def execute_algorithm(dados, categoricos):
    pass

dados = pd.read_csv(r"D:\tensorflowteste\excelfiles\export1.csv", sep=",")
categoricos = ['id_quote','dt_criacao','ds_tipo_obra','nm_prop_oportunidade', 'nm_ambiente','promocional','tipo_loja']


execute_algorithm(dados, categoricos)

Acurácia no conjunto de teste: 73.77%


,nm_prop_oportunidade,convertido,probabilidade_conversao
0,ana paula goncalves lemos,1,0.354008
1,fabiano martins werutsky,0,0.489150
2,juliana westphalen gonçalves,1,0.366401
